<a href="https://colab.research.google.com/github/peculab/EduData/blob/main/Example2_DataClearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

金門縣政府提供教育相關數據

https://drive.google.com/drive/folders/1ZnEmLN_Gbx074-dv_8m3chn2vvhaeUSK?usp=sharing

金門縣康軒國中小使用數據

https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing

In [27]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [42]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing')

In [43]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('anonymized_xapi').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,id,json,timestamp
0,281744,"{""id"": ""5d3ddc37-c43c-4342-8015-c1697ac5148b"",...",6/11/24 22:17
1,281746,"{""id"": ""b27139a8-101a-4d94-9f24-0fb6be429de7"",...",6/11/24 22:17
2,281747,"{""id"": ""73d06d71-56b5-4fd2-a951-7f3e485294ba"",...",6/11/24 22:17
3,281748,"{""id"": ""3c0c29d1-c8f9-441f-a820-bafd90e0fc58"",...",6/11/24 22:17
4,281749,"{""id"": ""6cce33e0-b8d4-47f4-a00d-6dd335c409c2"",...",6/11/24 22:17


In [60]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
alldays = max(set(df['timestamp'].dt.date)) - min(set(df['timestamp'].dt.date))

In [63]:
alldays.days

37

In [45]:
len(df)

42928

In [46]:
import json
df_expanded = pd.json_normalize(df['json'].apply(json.loads))

In [47]:
type(df_expanded['timestamp'][0])

str

In [48]:
df_expanded['id'] = df['id']
df_expanded['timestamp'] = pd.to_datetime(df['timestamp'])
df_expanded['Date'] = df_expanded['timestamp'].dt.date    # 提取日期部分
df_expanded['Time'] = df_expanded['timestamp'].dt.time    # 提取時間部分
df_expanded['Weekday'] = df_expanded['timestamp'].dt.day_name()  # 以星期幾的名稱表示（例如：Monday）
df_expanded.head()

,id,timestamp,actor.objectType,actor.name,actor.account.homePage,actor.account.name,verb.id,verb.display.zh-TW,object.objectType,object.id,...,context.team.mailto,context.team.mbox,object.definition.extensions.https://w3id.org/xapi/acrossx/extensions/alignment,object.definition.name.zh-TW,object.definition.interactionType,object.definition.correctResponsesPattern,object.definition.choices,Date,Time,Weekday
0,281744,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
1,281746,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
2,281747,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
3,281748,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
4,281749,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday


In [51]:
grouped = df_expanded.groupby(['actor.name', 'Date']).agg({
    'timestamp': list,
    'verb.display.zh-TW': list,
    'object.id': list
}).reset_index()

In [54]:
grouped['Date'] = pd.to_datetime(grouped['Date'])
grouped['Weekday'] = grouped['Date'].dt.day_name()  # 以星期幾的名稱表示（例如：Monday）
grouped

,actor.name,Date,timestamp,verb.display.zh-TW,object.id,Weekday
0,學生298,2024-06-11,"[2024-06-11 22:17:00, 2024-06-11 22:17:00, 202...","[觀看, 暫停, 觀看, 暫停, 觀看, 暫停, 開始嘗試]",[https://ksjh-km.kschool.com.tw/video-player/4...,Tuesday
1,學生298,2024-06-12,"[2024-06-12 11:39:00, 2024-06-12 11:46:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
2,學生298,2024-06-17,"[2024-06-17 13:39:00, 2024-06-17 13:48:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
3,學生298,2024-06-19,"[2024-06-19 11:16:00, 2024-06-19 11:51:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
4,學生298,2024-06-24,"[2024-06-24 13:47:00, 2024-06-24 14:04:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
...,...,...,...,...,...,...
868,學生590,2024-06-25,"[2024-06-25 13:06:00, 2024-06-25 13:07:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://saps-km.kschool.com.tw/student-quiz-a...,Tuesday
869,學生591,2024-06-25,"[2024-06-25 13:07:00, 2024-06-25 13:09:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://saps-km.kschool.com.tw/student-quiz-a...,Tuesday
870,學生592,2024-06-28,"[2024-06-28 15:09:00, 2024-06-28 15:10:00]","[查看, 查看]",[https://saps-km.kschool.com.tw/teacher-readin...,Friday
871,學生593,2024-07-02,[2024-07-02 11:21:00],[查看],[https://ksjh-km.kschool.com.tw/teacher-readin...,Tuesday


In [55]:
len(set(grouped['actor.name'])) #有297位學生

297

根據每位學生的瀏覽日期，來去畫出不同學生瀏覽影片的天數

In [67]:
student_days = grouped.groupby('actor.name')['Date'].nunique()
student_days.colnames = ['actor.name', 'Date']
student_days = student_days.reset_index()
student_days

,actor.name,Date
0,學生298,5
1,學生299,9
2,學生300,11
3,學生301,8
4,學生302,11
...,...,...
292,學生590,1
293,學生591,1
294,學生592,1
295,學生593,1


In [74]:
student_days[student_days['Date'] > (alldays.days / 4)]['actor.name']

,actor.name
2,學生300
4,學生302
5,學生303
6,學生304
7,學生305
9,學生307
11,學生309
13,學生311
14,學生312
15,學生313


In [71]:
# 計算每個 Date 值出現多少人
date_counts = student_days['Date'].value_counts().reset_index()
date_counts.columns = ['Date', 'Count']

# 使用 Plotly 繪製條形圖
fig = px.bar(date_counts.sort_values(by='Date'),
             x='Date', y='Count',
             labels={'Date': '天數', 'Count': '人數'},
             title='每個天數對應的人數')

# 顯示圖表
fig.show()

In [58]:
grouped[grouped['actor.name'] == '學生315']

,actor.name,Date,timestamp,verb.display.zh-TW,object.id,Weekday
164,學生315,2024-06-12,"[2024-06-12 08:40:00, 2024-06-12 08:57:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
165,學生315,2024-06-13,"[2024-06-13 12:25:00, 2024-06-13 12:55:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Thursday
166,學生315,2024-06-14,"[2024-06-14 12:24:00, 2024-06-14 12:36:00, 202...","[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
167,學生315,2024-06-17,"[2024-06-17 12:27:00, 2024-06-17 12:59:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
168,學生315,2024-06-18,"[2024-06-18 12:28:00, 2024-06-18 12:53:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday
169,學生315,2024-06-19,"[2024-06-19 07:26:00, 2024-06-19 07:43:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Wednesday
170,學生315,2024-06-21,"[2024-06-21 12:30:00, 2024-06-21 13:10:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Friday
171,學生315,2024-06-23,"[2024-06-23 15:47:00, 2024-06-23 15:47:00, 202...","[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Sunday
172,學生315,2024-06-24,"[2024-06-24 12:31:00, 2024-06-24 13:12:00, 202...","[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Monday
173,學生315,2024-06-25,"[2024-06-25 12:24:00, 2024-06-25 12:50:00, 202...","[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回...",[https://ksjh-km.kschool.com.tw/student-quiz-a...,Tuesday


In [78]:
grouped[grouped['actor.name'] == '學生315']['verb.display.zh-TW']

,verb.display.zh-TW
164,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
165,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
166,"[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回..."
167,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
168,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
169,"[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回..."
170,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
171,"[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回..."
172,"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答,..."
173,"[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回..."


In [79]:
grouped[grouped['actor.name'] == '學生315']['verb.display.zh-TW'].value_counts()

,count
verb.display.zh-TW,
"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",4
"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",2
"[開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",1
"[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",1
"[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 開始嘗試, 回答, 回答, 回答, ...]",1
"[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",1
"[開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, ...]",1
"[開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成, 開始嘗試, 開始嘗試, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 回答, 完成]",1


In [ ]:
grouped['unique_verbs'] = grouped['verb.display.zh-TW'].apply(lambda x: set(x))
unique_types = grouped['unique_verbs'].explode()
distinct_types = set(unique_types)
distinct_types

{'回答', '完成', '暫停', '查看', '觀看', '開始嘗試'}

在上述的數據清理與分析過程中，我們利用了來自金門縣康軒國中小的匿名學習行為數據，並針對學生使用平台的時間與行為進行了探討。

首先，透過 Google Sheets API，我們將資料載入 Pandas DataFrame 進行清理和格式化，進一步將資料轉換為標準的時間戳格式，並提取了其中的日期、時間及星期幾等資訊，以利後續的時間序列分析。

我們對每位學生的行為進行了聚合，計算他們在不同日期進行的操作次數，並分析了學生使用平台的行為動詞（如“回答”、“完成”、“暫停”、“查看”、“觀看”、“開始嘗試”），這些行為動詞描述了學生在平台上的具體學習活動。

根據資料，我們發現共有 297 名學生參與了平台使用，並根據每位學生的瀏覽日期，計算出學生在不同天數瀏覽影片的頻率。

進一步的分析中，我們計算了每位學生在所有天數內進行學習活動的總天數，並篩選出超過全體總天數的四分之一的學生，找出了持續活躍於平台的學生群體。此分析提供了學生學習參與的長期趨勢。

最後，透過 Plotly 繪製的條形圖，我們視覺化了每個天數中使用平台的人數分佈，顯示不同天數對應的活躍學生數量，以幫助教育機構進一步了解學生的學習習慣與學習高峰期。

這些結果可以為教育管理者提供有價值的洞見，幫助他們理解學生的使用情況，並進一步優化平台內容與功能，以提升學生的學習體驗和成效。

在這段數據分析中，我們發現了幾個值得注意的現象：

    學生活躍度的分佈：根據每位學生在不同日期的瀏覽紀錄，我們發現總共有 297 名學生使用該學習平台。但並非所有學生的使用頻率都相同，部分學生的活躍度明顯高於其他人。例如，篩選後發現有些學生的使用頻率超過了所有天數的四分之一，這意味著這些學生在平台上有著較為穩定和頻繁的學習行為。

    學習行為的多樣性：透過分析學生的學習動作（例如“回答”、“完成”、“暫停”、“查看”、“觀看”、“開始嘗試”），我們觀察到學生在使用平台時，進行了多樣化的操作行為，這代表學生在平台上的學習過程不是單一行為，而是存在多階段的互動（如觀看影片、暫停、回答問題等）。這樣的多階段學習行為可能代表學生的學習過程比較深入，而不是僅僅瀏覽或觀看影片。

    學習參與的時間分佈：在分析了學生學習活動的天數後，我們發現不同學生的學習參與天數有明顯差異。有些學生僅在少數天數內使用平台，而另一些學生則在較長的時間內保持活躍。這可能暗示部分學生只在特定時間或事件（例如考試或作業期限）前集中使用平台，而其他學生則更可能是持續學習。

    高峰期現象：從條形圖中，我們能夠清楚看到每個天數對應的人數分佈，這可以幫助我們發現學生使用平台的高峰期。這些高峰期很可能與學校活動、考試或其他學習相關的節點有關。例如，當天數中有明顯的活躍人數激增，可能反映出課程或作業的截止日期，這對於管理者而言是有用的資訊，可以幫助他們在適當時機提供更多學習資源或支援。

這些現象為教育機構提供了學生使用平台的行為模式，有助於針對不同的學習習慣做出相應調整，進一步提高學生的學習效果。